# Golf Swing Sequence Detection and Analysis
This notebook explores approaches to automating extractions of golf swing sequences from a golf range recordings.
I have been learning golf for some time, and recording my driving range session and reviewing the swings has been a very helpful way to improve.

Currently, I use tiktok or capcut to manually edit out swings from a full recording (Example is the video that I am using - "input_videos/ex.mp4") but I want to see if I can do this automatically using ML and computer vision techniques. So, let's dive in!

## Objectives of this notebook
- I need a way to find beginning and end frame of a golf swing sequence reliably
- this method needs to be tested on multiple videos that I have

In [37]:
from dotenv import load_dotenv
load_dotenv() 

True

I am using Roboflow's object detection model that has been trained to classify 12 different golf-specific objects from golf ball, golf club, club handle and more. I will probably only look at golf ball, golf club and golf club head. The reason is that only those classes have been detected when inferred on my video recording. 

I believe that simply analyzing the bounding boxes of the golf club head and golf ball can be an effective way to detect different stages of a golf swing sequence.

In [6]:
# import the InferencePipeline interface
from inference import InferencePipeline
# import a built-in sink called render_boxes (sinks are the logic that happens after inference)
from inference.core.interfaces.stream.sinks import render_boxes
from inference.core.interfaces.camera.entities import VideoFrame
import os
import json
from pprint import pprint

TARGET_DIR = "predictions"
api_key = os.getenv("ROBOFLOW_API_KEY")

arr = []

def save_prediction(prediction: dict, video_frame: VideoFrame) -> None:
    # pprint(prediction)
    # pprint(video_frame)
    # print("------------------------------------")
  # with open(os.path.join(TARGET_DIR, f"test.txt"), "a") as f:
    arr.append(prediction)
  #   json.dump(prediction, f)
  #   json.dump(video_frame, f)
    

# create an inference pipeline object
pipeline = InferencePipeline.init(
    model_id="golf-49wbh/1", # set the model id to a yolov8x model with in put size 1280
    video_reference="input_videos/ex.mp4", # set the video reference (source of video), it can be a link/path to a video file, an RTSP stream url, or an integer representing a device id (usually 0 for built in webcams)
    on_prediction=save_prediction, # tell the pipeline object what to do with each set of inference by passing a function
    api_key=api_key, # provide your roboflow api key for loading models from the roboflow api
)
# start the pipeline
pipeline.start()
# wait for the pipeline to finish
pipeline.join()

In [10]:
len(arr)

794

I quickly want to check if the length of the array equals the number of frames when extracted using OpenCV

In [5]:
import cv2

def get_frame_count(video_path):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video opened successfully
    if not video.isOpened():
        print("Error: Could not open video.")
        return None
    
    # Get the total number of frames
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Release the video capture object
    video.release()
    
    return frame_count

# Example usage
video_path = 'input_videos/ex.mp4'
frame_count = get_frame_count(video_path)
print(f'Total number of frames: {frame_count}')


Total number of frames: 794


Now let's look at what each prediction contains

In [14]:
arr[1]

{'image': {'width': 720, 'height': 1280},
 'predictions': [{'x': 473.5,
   'y': 1154.0,
   'width': 299.0,
   'height': 118.0,
   'confidence': 0.6061546802520752,
   'class': 'club',
   'class_id': 0,
   'detection_id': 'b4563659-9743-48ab-ac76-249facfdbb61'},
  {'x': 603.0,
   'y': 1161.5,
   'width': 38.0,
   'height': 117.0,
   'confidence': 0.59730464220047,
   'class': 'club_head',
   'class_id': 1,
   'detection_id': '54387061-bdbc-4c2c-bc4b-9eb8415e3257'},
  {'x': 101.5,
   'y': 512.5,
   'width': 45.0,
   'height': 289.0,
   'confidence': 0.5574514865875244,
   'class': 'club',
   'class_id': 0,
   'detection_id': '8d8dec66-7d44-45b9-97f5-7b2c814c32bf'}]}

In [13]:
num_preds = [len(x['predictions']) for x in arr]
print(set(num_preds))

{0, 1, 2, 3, 4}


Let's see what types of classes have been detected. When I tried the model on Roboflow's browser, only golf club, golf club head and golf ball were detected, but I want to check if this is true

In [15]:
def get_unique_classes(data):
    unique_classes = set()
    for item in data:
        for prediction in item['predictions']:
            unique_classes.add(prediction['class'])
    return unique_classes

# Assuming your array is named 'data'
unique_classes = get_unique_classes(arr)
print("Unique classes:", unique_classes)

Unique classes: {'golfball', 'club', 'club_head'}


I now want to visualize these predictions using Plotly.

In [18]:
#!pip install plotly

In [31]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def create_trajectory_plot(data):
    # Create a list to hold all predictions with frame numbers
    all_predictions = []
    for frame_num, frame_data in enumerate(data):
        for pred in frame_data['predictions']:
            pred['frame'] = frame_num
            all_predictions.append(pred)
    
    # Create a DataFrame
    df = pd.DataFrame(all_predictions)
    
    # Get unique classes
    classes = df['class'].unique()
    
    # Create subplots: one for x-coordinate over time, one for y-coordinate
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                        subplot_titles=("X-coordinate Trajectory", "Y-coordinate Trajectory"))
    
    # Color map for classes
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    color_map = dict(zip(classes, colors[:len(classes)]))
    
    for cls in classes:
        class_data = df[df['class'] == cls].sort_values('frame')
        
        # Add trace for x-coordinate
        fig.add_trace(
            go.Scatter(x=class_data['frame'], y=class_data['x'], mode='lines+markers',
                       name=f'{cls} (x)', line=dict(color=color_map[cls])),
            row=1, col=1
        )
        
        # Add trace for y-coordinate
        fig.add_trace(
            go.Scatter(x=class_data['frame'], y=class_data['y'], mode='lines+markers',
                       name=f'{cls} (y)', line=dict(color=color_map[cls], dash='dash')),
            row=2, col=1
        )
    
    # Update layout
    fig.update_layout(height=800, title_text="Object Trajectories in Golf Swing Video")
    fig.update_xaxes(title_text="Frame Number")
    fig.update_yaxes(title_text="X Position", row=1, col=1)
    fig.update_yaxes(title_text="Y Position", row=2, col=1)
    
    return fig

# Assuming your array is named 'data'
fig = create_trajectory_plot(arr)
fig.show()

Let's save this dataframe that the plot function is using since it may be useful later on:

In [33]:
all_predictions = []
for frame_num, frame_data in enumerate(arr):
    for pred in frame_data['predictions']:
        pred['frame'] = frame_num
        all_predictions.append(pred)

# Create a DataFrame
df = pd.DataFrame(all_predictions)

The resulting plot provides a great visual representation of the object trajectories throughout the golf swing video. Here are some observations:
- Golf ball positions might be the best way to detect when the ball has been hit. However, it is not clear just from this example that golf ball can be reliably detected.
- Also, just looking at golf ball will be hard to find the beginning and the end of a swing sequence, which is all that we are interested in.
- I do see distinct patterns in the trajectories, likely corresponding to different phases of the golf swing.

Next, I want to see what each frame number looks like. To do so, I will need a tool to visualize different frame numbers